#### Creating data for sequential choice models
The series of models I plan to use in my thesis project, inspired by habitat-selection functions from movement ecology, use environmental covariates of available resources (e.g., distance from current location, turning angle from previous bearing angle, size/point value, etc.) to predict whether a resource will be chosen next. By specifying a series of models containing different parameters, I will fit different cognitive heuristic strategies to human sequential choice data and determine parameters of significance to the foraging strategies of participants. 

To start, I must take existing foraging data in the form of logs of collected objects during play and expand it out to include the covariates of interest of all available objects. This data needs to also reflect the removal and reintroduction of collected objects during the course of play. 

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# read in data
forage_data = pd.read_csv(
    "../../data/piloting/march24_pilot_forage_data_w_ids.csv"
)
all_lvls_obj_location_data = pd.read_csv(
    "../../data/arrangements/object-location-data.csv"
)

##### Adding covariates and all available objects
The two covariates of interest needed to be added to this expanded data set are distance from current location and turning angle 

In [52]:
# distance matrices store distances between coconuts
def create_distance_matrices(obj_locs=pd.DataFrame):

    # create list for levels
    dist_matrix_list = []

    # for each level, create matrix of all distances
    for level in range(0, 10):
        # filter for this level
        level_locs = obj_locs[obj_locs.level == "_level_"+str(level+1)]

        # reset index for consistent indexing
        level_locs.index = [*range(0, len(level_locs))]

        # create empty matrix
        level_m = np.zeros((len(level_locs), len(level_locs)), dtype=float)

        # fill this level's matrix
        for i in range(0, len(level_locs)):
            for j in range(0, len(level_locs)):
                # calc distance and store in matrix
                level_m[i, j] = np.sqrt(
                    np.pow(level_locs.x[i]-level_locs.x[j], 2) +
                    np.pow(level_locs.y[i]-level_locs.y[j], 2)
                )

        # add to list
        dist_matrix_list.append(level_m)

    return dist_matrix_list

In [ ]:
# calculate turning angles
def calculate_turning_angle(
    previous_head_angle=float, current_obj_id=int, obj_locs=pd.DataFrame
):

    # empty list for angles
    ta_list = []

    # loop through df and calculate each ta
    for obj in obj_locs.obj_ID:
        if obj == current_obj_id:
            ta_list.append(pd.NA)
        else:
            ta = np.atan2(
                obj_locs[obj_locs.obj_ID == obj].y[obj-1] -
                obj_locs[obj_locs.obj_ID ==
                         current_obj_id].y[current_obj_id-1],
                obj_locs[obj_locs.obj_ID == obj].x[obj-1] -
                obj_locs[obj_locs.obj_ID == current_obj_id].x[current_obj_id-1]
            ) - previous_head_angle

            ta_list.append(ta)

    return ta_list

In [127]:
lvls = create_distance_matrices(all_lvls_obj_location_data)

In [ ]:
# neighbors distance
def neighborhood_value(
    obj_ind=int, lvl_ind=int, lvl_matrices=list, num_neighbors=int, already_collected=list
):

    # filter for level distances
    lvl_dists = lvl_matrices[lvl_ind]

    # filter for already collected items
    if len(already_collected) > 0:
        available_indices = [
            x for x in [*range(0, len(lvl_dists[obj_ind]))] if x not in already_collect
        ]
        active_obj_dists = lvl_dists[obj_ind][available_indices]
    else:
        active_obj_dists = lvl_dists[obj_ind]

    return np.sum(np.reciprocal(sorted(active_obj_dists)[1:num_neighbors]))

In [ ]:
# combine together to create function to expand df
def expand_df_with_covariates(
    orig_df=pd.DataFrame,
    all_obj_locs_df=pd.DataFrame
):

    # create empty df to add to
    expanded_df = pd.DataFrame({
        'subject': int,
        'level': int,
        'collection_num': int,
        'obj_ID': int,
        'point_value': int,
        'distance': float,
        'turning_angle': float,
        'neighbor_value': float,
        'used': int,
        'time': float
    })

    # create df to track collections
    collect_df = pd.DataFrame({
        'obj_ID': int,
        'time_to_respawn': float
    })

    # create all distance matrices
    lvl_dist_matrices = create_distance_matrices(all_obj_locs_df)

    # loop through df and expand
    for i in range(0, len(orig_df)):

        # slice row
        row_of_interest = orig_df.iloc[i]

In [ ]:
for i in range(0, len(forage_data)):
    print(forage_data.iloc[i].obj_ID)
    break

32
